In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 
from scipy.sparse import csr_matrix
!pip install pymorphy2
import pymorphy2
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report




In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
file = open("../input/doc-cls/corp0.txt", "r")
filedata = file.readlines()
print("Размер dataset", len(filedata))

# dictionary = file.get_feature_names()
# print(" Размер словаря корпуса до предварительной обработки.", len(dictionary))


### 1. Подготовка набора данных.

Вариант токенизации:

·     ограничиваем максимальную длину документа (по числу словоформ);

·     приводим в нижний регистр;

·     иностранные слова заменяем на FRGN;

·     элементы с цифрами заменяем на NUMB;

·     удаляем знаки препинания;

·     Лемматизация русских слов. Её цель - сокращение словаря;

·     разделение данных и меток в разные файлы (x.txt, y.txt).

Лемматизация – это приведение словоформы к лемме, или нормальной форме. 

In [ ]:
#Проверка. Пример предварительной обработки корпуса с выводом результатов для нескольких элементов датасета

dataset = filedata [35:39]#[54:60]
print("Dataset \n",dataset, "\n")
print("Размер dataset", len(dataset))

#делаем все слова нижнего регистра
a = [x.lower() for x in dataset]

# иностранные слова заменяем на FRGN; элементы с цифрами заменяем на NUMB
b = []
n = len(a)
for i in range (n):
    b.append(a[i].split(" "))
    for j in range (0, len(b[i])):
        if (re.search('[a-z]', b[i][j])):
            b[i][j]= "frgn"#"FRGN"
        if (re.search('[0-9]', b[i][j])):
            b[i][j]= "numb"#"NUMB"
    b[i]=" ".join(b[i])
    
#print(b)

out = []
for x in b:  
     out.append (re.sub(r'[^\w\s]','', x))

dataset = out
print("--------------------------------------")
print("dataset после подготовки данных")
print(dataset)

  



In [ ]:
# лемматизация
# Заменяет  слова леммами 
def to_normal_form(morph, s): 
    s2 = s.split() # Список слов предложения s 
    s = '' 
    for w in s2: 
        w = morph.parse(w)[0].normal_form 
        s += (' ' + w) 
    return s.lstrip() 
morph = pymorphy2.MorphAnalyzer() 
for i in range (len(dataset)):
    dataset[i]=to_normal_form(morph, dataset[i])
print (dataset)
    

In [ ]:
pwd

In [ ]:
dataset = out

#извлекаем метки
print("--------------------------------------")
y_data = []
n = len (dataset)
for i in range (n):
    y_words = dataset[i].split(" ")
    y_data.append(y_words[0])
    
#print("Разбиение предложения по словам y_words", y_words,"\n")
print("Метки y_label, \n", y_data,"\n")
#извлекаем предложения без меток
print("--------------------------------------")
x_data = []
n = len (dataset)
for i in range (n):
    ind_begin = dataset[i].index(" ")
    ind_begin +=1
    x_data.append(dataset[i][ind_begin:])
    
print("Предложения без меток: \n", x_data,"\n")

f_x = open("f_x.txt", "w")
f_y = open("f_y.txt", "w")

np.savetxt(f_x, x_data, fmt='%s')
np.savetxt(f_y, y_data, fmt='%s')

f_x.close()
f_y.close() 


# x_data = np.array(x_data)
# y_data = np.array(y_data)

# for  line in x_data:
#      f_x.write(line+"\n")
# for  line in y_data:
#      f_y.write(line+"\n")
    

# Основная программа

### Проверка. Пример предварительной обработки корпуса с выводом результатов для нескольких элементов датасета

In [ ]:
# #Проверка. Пример предварительной обработки корпуса с выводом результатов для нескольких элементов датасета
# dataset = filedata 
# #print("Dataset \n",dataset, "\n")
# print("Размер dataset", len(dataset))

# #делаем все слова нижнего регистра
# a = [x.lower() for x in dataset]

# # иностранные слова заменяем на FRGN; элементы с цифрами заменяем на NUMB
# b = []
# n = len(a)
# for i in range (n):
#     b.append(a[i].split(" "))
#     for j in range (0, len(b[i])):
#         if (re.search('[a-z]', b[i][j])):
#             b[i][j]= "frgn"#"FRGN"
#         if (re.search('[0-9]', b[i][j])):
#             b[i][j]= "numb"#"NUMB"
#     b[i]=" ".join(b[i])
    
# #print(b)
# out = []
# for x in b:  
#      out.append (re.sub(r'[^\w\s]','', x))

# dataset = out
# # print("--------------------------------------")
# # print("dataset после подготовки данных")
# # print(dataset)
# print("Результат получен")

  



###  лемматизация

In [ ]:
# # лемматизация
# # Заменяет  слова леммами 
# def to_normal_form(morph, s): 
#     s2 = s.split() # Список слов предложения s 
#     s = '' 
#     for w in s2: 
#         w = morph.parse(w)[0].normal_form 
#         s += (' ' + w) 
#     return s.lstrip() 
# morph = pymorphy2.MorphAnalyzer() 
# for i in range (len(dataset)):
#     dataset[i]=to_normal_form(morph, dataset[i])
# #print (dataset)
# print("Результат получен")
    

In [ ]:
# dataset = out

# #извлекаем метки
# y_data = []
# n = len (dataset)
# for i in range (n):
#     y_words = dataset[i].split(" ")
#     y_data.append(y_words[0])
    
# #print("Разбиение предложения по словам y_words", y_words,"\n")
# #print("Метки y_label, \n", y_data,"\n")
# #извлекаем предложения без меток
# x_data = []
# n = len (dataset)
# for i in range (n):
#     ind_begin = dataset[i].index(" ")
#     #ind_end = dataset[i].index("\n")
#     ind_begin +=1
#     #x_data.append(dataset[i][ind_begin:ind_end])
#     x_data.append(dataset[i][ind_begin:])
    
# #print("Предложения без меток: \n", x_data,"\n")

# f_x = open("f_x.txt", "w")
# f_y = open("f_y.txt", "w")

# np.savetxt(f_x, x_data, fmt='%s')
# np.savetxt(f_y, y_data, fmt='%s')

# f_x.close()
# f_y.close() 
# print("Данные записаны в файлы")
    

### 2. Делим набор данных на обучающее  и проверочное множества.

In [ ]:
x_file = open("../input/new-data/f_x_dat.txt", "r")
x_data = x_file.readlines()
print("Размер dataset", len(x_data))

y_file = open("../input/new-data/f_y_dat.txt", "r")
y_data = y_file.readlines()
print("Кол-во меток", len(y_data))


#<загрузка  данных и меток в списки (массивы) x, y>
k_split = 0.2

x = x_data
y = y_data

from sklearn.preprocessing import LabelEncoder
import keras
lencoder = LabelEncoder()
lencoder.fit(y_data)

x_trn, x_vl, y_trn, y_vl = train_test_split(x, y, test_size = k_split, shuffle = True)
print( "кол-во данных в x_trn", len(x_trn))
print( "кол-во данных в x_ vl", len(x_vl))
#print(x_trn,"\n", x_vl,"\n", y_trn,"\n", y_vl)



### 3. Векторизация.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 
vec = CountVectorizer(token_pattern = '\w+', binary = False) # Если задать ,binary = True то векторы, представляющие строки текста, будут бинарными
len_trn = len(x_trn) 
x_trn.extend(x_vl) # Объединяем x_trn и x_vl и получаем полный корпус 
x_trn = vec.fit_transform(x_trn) 
x_vl = x_trn[len_trn:] 
x_trn = x_trn[:len_trn] 
# Преобразуем разреженные матрицы в массивы 
x_trn = np.float32(x_trn.toarray()) 
x_vl = np.float32(x_vl.toarray())
print(x_trn.shape)
print(x_vl.shape)

print(len(vec.get_feature_names()))

In [ ]:
import time

### SGDClassifier.

In [ ]:
from sklearn.linear_model import SGDClassifier 
doc_clf = SGDClassifier(loss = 'hinge', max_iter = 1000, tol = 1e-3) 

start_time = time.time()
#print(y_trn)
doc_clf.fit(x_trn, y_trn) # Обучение классификатора 
end_time = time.time() - start_time
print("Время выполнения :  {:.8f} секунд " .format(end_time) )
print('Оценка точности классификации') 
score = doc_clf.score(x_vl, y_vl) 
print('Точность на проверочном множестве:', round(score, 4)) 
score = doc_clf.score(x_trn, y_trn) 
print('Точность на обучающем множестве:', round(score, 4))

predictions = doc_clf.predict(x_vl)
print(classification_report(lencoder.transform(y_vl), lencoder.transform(predictions), digits = 4, target_names = lencoder.classes_))



### LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression 
doc_clf = LogisticRegression(solver = 'lbfgs', # newton-cg 
                             max_iter = 500, multi_class = 'auto') 
start_time = time.time()
doc_clf.fit(x_trn, y_trn) # Обучение классификатора 
end_time = time.time() - start_time
print("Время выполнения :  {:.8f} секунд " .format(end_time) )
print('Оценка точности классификации') 
score = doc_clf.score(x_vl, y_vl) 
print('Точность на проверочном множестве:', round(score, 4)) 
score = doc_clf.score(x_trn, y_trn) 
print('Точность на обучающем множестве:', round(score, 4))

predictions = doc_clf.predict(x_vl)
print(classification_report(lencoder.transform(y_vl), lencoder.transform(predictions), digits = 4, target_names = lencoder.classes_))


In [ ]:
#---------------------
from sklearn.svm import SVC 
doc_clf_svc = SVC()
#doc_clf = SGDClassifier(loss = 'hinge', max_iter = 1000, tol = 1e-3) 

start_time = time.time()
#print(y_trn)
doc_clf_svc.fit(x_trn, y_trn) # Обучение классификатора 
end_time = time.time() - start_time
print("Время выполнения :  {:.8f} секунд " .format(end_time) )
print('Оценка точности классификации') 
score = doc_clf_svc.score(x_vl, y_vl) 
print('Точность на проверочном множестве:', round(score, 4)) 
score = doc_clf_svc.score(x_trn, y_trn) 
print('Точность на обучающем множестве:', round(score, 4))

predictions = doc_clf.predict(x_vl)
print(classification_report(lencoder.transform(y_vl), lencoder.transform(predictions), digits = 4, target_names = lencoder.classes_))